# Part 3 - Text analysis and ethics

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [1]:
import pandas as pd
from nltk.tag import pos_tag
from nltk import RegexpParser
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()

In [2]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load stopwords
sw = set(stopwords.words('english'))

In [4]:
# sw

In [5]:
'''We want to find the alternative forms of stopwords that have the "'" symbol in them 
in order to be able to add also to stopwords the word without this symbol'''

pattern = r'\w+\'\w+'

new_stopwords = []
for word in sw:
    # If it finds a word that contains "'" it appends the word in new_stopwords list
    if len(re.findall(pattern,word)) == 1:
        new_stopwords.append(re.findall(pattern,word)[0].replace('\'',''))
# new_stopwords

In [6]:
# After checking those "new" words we add them to the stopwords variables named sw
for word in new_stopwords:
    sw.add(word)
# sw

In [7]:
basedir = os.getcwd()
df = pd.read_csv(os.path.join(basedir,'reviews.csv'))
# deal with empty reviews
df.comments = df.comments.fillna('')

In [8]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [9]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [10]:
def process_reviews(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    # Initialize 3 lists one for each column we will create
    tokenized_col = []
    tagged_col = []
    lower_tagged_col = []


    mylen = len(df)
    count = 0
    
    # Iterate through the given dataframe
    for index, row in df.iterrows():
        # tokenize the words for the comments of a row
        token = word_tokenize(row.comments)
        # Append the tokenized words to the proper list
        tokenized_col.append(token)
        # Tag the tokenized words of the row and then append them to the proper list
        tagged_col.append(pos_tag(token))
        # lower_tagged.append(list(set(pos_tag([item.lower() for item in token]))))
        # Make the tagged words lowercased and then if they are not stopwords append them to the lower_tagged_col list
        lower_tagged_col.append(pos_tag([item.lower() for item in token if item.lower() not in sw]))
        count += 1

        if count % 50000 == 0: print(f'{count} out of {mylen}')

    # Set as values of the 3 new columns the proper list we created for each one
    df['tokenized'] = tokenized_col
    df['tagged'] = tagged_col
    df['lower_tagged'] = lower_tagged_col

    # Return the modified dataframe
    return df

In [11]:
df = process_reviews(df)

50000 out of 452143
100000 out of 452143
150000 out of 452143
200000 out of 452143
250000 out of 452143
300000 out of 452143
350000 out of 452143
400000 out of 452143
450000 out of 452143


### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [13]:
def get_vocab(df):
    '''
    Based on the lower_tagged column of the dataframe (df) that this function receives it creates a vocabulary of 
    ‘center’ (the x in the PMI equation) and ‘context’ (the y in the PMI equation) words. 
    The vocabulary of center words will be the 1,000 most frequent NOUNS (words with a PoS tag starting with ‘N’), 
    and the context words will be the 1,000 most frequent words tagged as either VERB or ADJECTIVE 
    (words with any PoS tag starting with either ‘J’ or ‘V’).

    Args:
        df: The dataframe we want to modify.

    Returns: The ‘center’ and ‘context’ vocabularies as lists.
    '''
    
    # Initialize 2 empty lists for the 2 vocabularies to be filled
    cent_list, cont_list = [], []

    # Iterate through the 'lower_tagged' column of the df provided
    for review in df.lower_tagged:
        
        '''For every word in a review (a record in the 'lower_tagged' column) that the condition is true for either 
        center or context list is appended to the appropriate list '''
        cent_list.extend([word for word in [list_of_words[0] for list_of_words in review if list_of_words[1][0] == 'N']])
        cont_list.extend([word for word in [list_of_words[0] for list_of_words in review if (list_of_words[1][0] == 'J') 
                                            or (list_of_words[1][0] == 'V')]])
    
    # We create 2 dictionaries that holds information about the frequency of the words in the 2 lists we have
    cent_dict = Counter(cent_list)
    cont_dict = Counter(cont_list)

    # We sort the dictionaries based on their value for frequency of the words and then keep the 1000 most frequent in each list
    cent_vocab = [key for key, value in sorted(cent_dict.items(), key=lambda item: item[1], reverse=True)][:1000]
    cont_vocab = [key for key, value in sorted(cont_dict.items(), key=lambda item: item[1], reverse=True)][:1000]

    # Return the lists
    return cent_vocab, cont_vocab

In [14]:
cent_vocab, cont_vocab = get_vocab(df)

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [19]:
def get_coocs(df, cent_vocab, cont_vocab):
    '''
    This function with the 1,000-word vocabularies of center and context words, creates a co-occurrence matrix 
    where, for each center word, we keep track of how many of the context words co-occur with it. 

    Args:
        df: The dataframe that holds the reviews and we will base the creation of co-occurence matrix.
        cent_vocab: The vocabulary of center words.
        cont_vocab: The vocabulary of context words.

    Returns: A co-occurrence matrix of center and context words.
    '''
    
    sentences = [] 
    words_in_sentence = []
    start = pd.to_datetime('today')
    diff = 0
    count = 0
    for index, row in df.iterrows():
        count += 1
        for (word, tag) in row.lower_tagged:            
            if tag[0] == '.':
                sentences.append(words_in_sentence)
                words_in_sentence = []
            else:
                words_in_sentence.append((word, tag))
        end = pd.to_datetime('today')
        diff = (end-start).total_seconds()
        if count % 25000 == 0 : print(f'{count} -- {diff/60}')
    
    print(f'yolo in {diff/60} minutes')
    
    start = pd.to_datetime('today')
  
    '''Create a dict where we have as keys the 1,000 center_words and as value the sentences they occur.
       In order to Filter the sentences we call the Filter() function'''
    sentences_per_center_word = {center_word : Filter(sentences, center_word) for center_word in cent_vocab}

    # return sentences_per_center_word
    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    print(f'swag in {diff/60} minutes')
    
    # Initialize an empty dictionary for the co-occurence matrix
    coocs = {}

    count = 0
    count2 = 0
    diff = 0
    
    # Iterate through the dictionary that keeps the sentences for each center_word
    for center_word, sentences in sentences_per_center_word.items():
        # Initialize an empty list for the context words that co-occur with the center_word
        words = []
        count += 1
        start = pd.to_datetime('today')
        count2 = 0
        
        # Iterate through the sentences the center_word occurs
        for sentence in sentences:
            count2 += 1
            
            # Create a list with the context words that co-occur with the center_word            
            words_of_sentence = [word for (word, tag) in sentence if word in cont_vocab and (( tag[0] == 'J' ) or ( tag[0] == 'V' ))]
            '''If the list we created above is not empty (it was succesful in searching for contenxt words) 
            extend the words list with it.'''
            if len(words_of_sentence) > 0: words.extend(words_of_sentence)
                
        end = pd.to_datetime('today')
        diff += (end-start).total_seconds()
        
        coocs[center_word] = dict(Counter(words))
        
        if count % 200 == 0 : print(f'{count} center_word out of {len(sentences_per_center_word)} in {diff/60} minutes')

            
    print(diff/60)
    
    return coocs 

def Filter(sentences, center_word):
    '''
    This function receives all the sentences in the comments column and 
    creates a list of the sentences the center_word occurs.

    Args:
        sentences: A list of all the sentences in the comments.
        center_word: A center word to find its sentences.

    Returns: A list of the sentences that the center_word occurs.
    '''
    
    # Initialize a list to collect the sentences a center_word is in.
    start = pd.to_datetime('today')
    
    sentences_for_center_word = [sentence for sentence in sentences 
                                 if len([word for (word, tag) in sentence if center_word == word and tag[0] == 'N']) > 0]
    end = pd.to_datetime('today')
    diff = (end-start).total_seconds()
    
    # print(f'{center_word} in {diff} seconds')
    return sentences_for_center_word

In [21]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

25000 -- 0.1677937
50000 -- 0.27662600000000004
75000 -- 0.3838142833333333
100000 -- 0.54413095
125000 -- 0.6599908666666666
150000 -- 0.7669695333333333
175000 -- 0.8743629833333333
200000 -- 0.9911998833333333
225000 -- 1.16649735
250000 -- 1.2794043166666667
275000 -- 1.3897007333333333
300000 -- 1.54946645
325000 -- 1.6757442
350000 -- 1.7869442666666666
375000 -- 2.056790116666667
400000 -- 2.1917908833333333
425000 -- 2.2999646166666663
450000 -- 2.4086426333333337
yolo in 2.4189175166666668 minutes
place in 1.625831 seconds
apartment in 1.364454 seconds
location in 1.346062 seconds
stay in 1.410675 seconds
amsterdam in 1.441394 seconds
host in 1.419703 seconds
everything in 1.518561 seconds
city in 1.617694 seconds
room in 1.455175 seconds
time in 1.554505 seconds
house in 1.514641 seconds
area in 1.41655 seconds
home in 1.606361 seconds
très in 1.488691 seconds
center in 2.758859 seconds
restaurants in 2.185885 seconds
’ in 1.971061 seconds
station in 2.074838 seconds
minutes 

### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [24]:
def cooc_dict2df(coocs):
    '''
    This function takes as input the dictionary of co-occurence matrix for center and context words. 
    It converts the dictionary to a 1000x1000 pandas DataFrame.

    Args:
        coocs: The dictionary of co-occurence matrix for center and context words.

    Returns: A 1000x1000 pandas DataFrame.
    '''
    
    # Initialize a pandas DataFrame with columns the context words and indexes the center words
    coocdf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

    # Iterate through the dataframe we created previously to fill it with values
    for index, row in coocdf.iterrows():
        for word in cont_vocab:
            ''' If the pair of index(center word) and word (context word) co-occurs 
                it will add the value to proper place in the dataframe.
                Otherwise the Error of coocs not having a value for this pair will be caught 
                and the value of 0 will added to the corresponding place.'''
            
            try:
                coocdf[word][index] = coocs[index][word]
            except: 
                coocdf[word][index] = 0

    # Return the pandas DataFrame
    return coocdf

In [25]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [26]:
def cooc2pmi(df):
    '''
    This function converts the raw co-occurence counts pandas DataFrame 
    to a DataFrame that keeps the information for the PMI scores.

    Args:
        df: The dataframe we want to convert from raw co-occurence counts to PMI scores.

    Returns: A pandas DataFrame with PMI scores for the pairs of center and context words.
    '''
    
    pmidf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

    N = 0
    for index, row in df.iterrows():
        N += sum(row)
    
    count = 0
    for index, row in df.iterrows():
        for word in cont_vocab:
            try:
                pmi = df[word][index] / (sum(df[word])/N / sum(row)/N)
                if pmi == 0:
                    pmidf[word][index] = 0
                else:
                    pmidf[word][index] = np.log([pmi])[0] 
            except: 
                pmidf[word][index] = 0
      
    return pmidf

In [27]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

(1000, 1000)

### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [29]:
def topk(df, center_word, N=10):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        center_word: The dataframe we want to modify.
        N: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    dicts_ = {word: df[word][center_word] for word in cont_vocab}
    top_words = [key for key, value in sorted(dicts_.items(), key=lambda item: item[1], reverse=True)][:N]

    return top_words

In [30]:
topk(pmidf, 'place')

['recommand',
 'recomend',
 'recommend',
 'reccomend',
 '’',
 '‘',
 'looking',
 'spotless',
 'recommendable',
 'magical']

In [31]:
topk(pmidf, 'location')

['prime',
 'brilliant',
 'fantastic',
 'great',
 'perfect',
 'superb',
 'terrific',
 'excellent',
 'fabulous',
 'better']

In [32]:
topk(pmidf, 'coffee')

['tea',
 'nespresso',
 'microwave',
 'complimentary',
 'supplied',
 'fridge',
 'stocked',
 'cheese',
 'fresh',
 'drink']

In [33]:
topk(pmidf, 'stay')

['enjoyed',
 'enjoyable',
 'pleasant',
 'overall',
 'hesitate',
 'memorable',
 'love',
 'pleased',
 'ensure',
 'satisfied']

In [34]:
topk(pmidf, 'petits')

['aux', 'des', 'sont', 'deux', 'avec', 'apprécié', 'et', 'ses', 'les', 'ont']

In [35]:
topk(pmidf, 'sauber')

['wohnung',
 'sehr',
 'zimmer',
 'das',
 'alles',
 'und',
 'bad',
 'ist',
 'wie',
 'gute']

# 3.b Ethical, social and legal implications



Local authorities in touristic hotspots like Amsterdam, NYC or Barcelona regulate the price of recreational apartments for rent to, among others, ensure that fair rent prices are kept for year-long residents. Consider your price recommender for hosts in Question 2c. Imagine that Airbnb recommends a new host to put the price of your flat at a price which is above the official regulations established by the local government. Upon inspection, you realize that the inflated price you have been recommended comes from many apartments in the area only being offered during an annual event which brings many tourists, and which causes prices to rise. 

In this context, critically reflect on the compliance of this recommender system with **one of the five actions** outlined in the **UK’s Data Ethics Framework**. You should prioritize the action that, in your opinion, is the weakest. Then, justify your choice by critically analyzing the three **key principles** outlined in the Framework, namely _transparency_, _accountability_ and _fairness_. Finally, you should propose and critically justify a solution that would improve the recommender system in at least one of these principles. You are strongly encouraged to follow a scholarly approach, e.g., with peer-reviewed references as support. 

Your report should be between 500 and 750 words long.  

### Your answer here. No Python, only Markdown.

Write your answer after the line.

---

The weakest action from the five outlined in the UK’s Data Ethics Framework is the “Review the quality and limitations of the data”. The bias of the prices for apartments only being offered during an annual event was not assessed and it creates false outputs and that has a negative effect to people. On the one hand the guests have a false opinion about prices as they are inflated by an event that occurs one time per year whilst this recommendation system is being used for the whole year by the guests. That means that they might select a room that is more expensive than it should be. On the other hand, the hosts that want to offer a new room may become too optimistic and set a price per night for their room that will mean guests will not choose and they will finally have a loss of income due to false information about the market status. As it is right now, we have a serious problem of Transparency to our data. It is not clear that we have a bias in our dataset that inflates prices and causes major problem to the people using our recommendation system. About the Accountability of our project there are also major concerns.  The challenge is big and it is described by Floridi and Taddeo in their journal “some crucial challenges include moral responsibility and accountability of both designers and data scientists with respect to unforeseen and undesired consequences as well as missed opportunities. Unsurprisingly, the ethical design and auditing of algorithms’ requirements and the assessment of potential, undesirable outcomes (e.g. discrimination or the promotion of antisocial content) is attracting increasing research.” [1]. We have not established any mechanism to combat biases against events price inflation or any other bias that could harm people or neighbourhoods. There is a chance that there might be a discrimination against some neighbourhoods, and they have lower price than they should. This statement also applies to the principle of Fairness. There has been no research as to if host that are people of colour for example tend to have lower prices to their apartments due to failures in our recommendation system to combat this discrimination. As stated by Markham 2018, “it is impossible to standardize or universalize what constitutes the ethically correct actions in technology design and research contexts, not least because we cannot predict what will happen as a result of our choices.”[2]. So, we have to implement a custom solution for our project that will assess biases and if possible, eliminate the discrimination against either guests or hosts. There has to be an extensive search on how the prices have been established at this level for each type of accommodation and for the neighbourhoods. Furthermore, we have to take into account the trends on the selection of an accommodation based on the profile of the guests but also the hosts. All these mechanism have to be transparent so everybody can understand the logic behind every decision the team that created this project made.

## References

[1] What is data ethics?, Luciano Floridi & Mariarosaria Taddeo, https://royalsocietypublishing.org/doi/10.1098/rsta.2016.0360 

[2] Ethics as Methods: Doing Ethics in the Era of Big Data Research, Annette N Markham, Katrin Tiidenberg and Andrew Herman, https://journals.sagepub.com/doi/pdf/10.1177/2056305118784502 
